In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [2]:
from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.fgw import encode_target

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [3]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"

In [4]:
molopt, config = load_model(args.output_dir + "model_gcn3_123", MolOpt, args.device)
molopt, config

(MolOpt(
   (GCN): GCN(
     (W_message_i): Linear(in_features=100, out_features=50, bias=False)
     (W_message_h): Linear(in_features=50, out_features=50, bias=False)
     (W_message_o): Linear(in_features=143, out_features=50, bias=True)
     (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
     (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
     (dropout_gcn): Dropout(p=0.0, inplace=False)
     (dropout_ffn): Dropout(p=0.0, inplace=False)
   )
   (opt0): Linear(in_features=50, out_features=50, bias=True)
   (opt1): Linear(in_features=50, out_features=50, bias=True)
 ),
 Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', init_decoder_model='gcn3_decode', init_model='gcn3', linear_out=False, n_epochs=1, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, output_dir='mol_opt/output', pc_hidden=50, pred_hidden=50))

In [5]:
molopt_decoder, config_decoder = load_model(args.output_dir + "model_gcn3_decode_123", MolOptDecoder, args.device)
molopt_decoder, config_decoder

(MolOptDecoder(
   (fc1_SYMBOLS): Linear(in_features=50, out_features=50, bias=True)
   (fc2_SYMBOLS): Linear(in_features=50, out_features=64, bias=True)
   (fc1_CHARGES): Linear(in_features=50, out_features=50, bias=True)
   (fc2_CHARGES): Linear(in_features=50, out_features=5, bias=True)
   (fc1_BONDS): Linear(in_features=100, out_features=50, bias=True)
   (fc2_BONDS): Linear(in_features=50, out_features=5, bias=True)
 ),
 Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', init_decoder_model='gcn3_decode', init_model='gcn3', linear_out=False, n_epochs=1, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, output_dir='mol_opt/output', pc_hidden=50, pred_hidden=50))

In [6]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 48, True)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)

In [7]:
for i in val_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

# Assess GCN

In [8]:
x_embedding, x_delta_hat = molopt.forward(X)
y_embedding = molopt.encode(Y)

In [9]:
x_embedding + x_delta_hat

tensor([[  54.4833,  -92.9698,   69.4455,  ...,  -48.8200,  -36.5185,
           46.4153],
        [  27.2280,  -87.4215,   63.3841,  ...,  -97.9667,  -75.4884,
           52.1544],
        [  38.0533,  -83.1627,   66.2401,  ...,  -81.5087,  -53.6912,
           45.2554],
        ...,
        [  72.5814, -111.4858,   86.8763,  ...,  -38.2057,  -27.3535,
           47.9529],
        [  19.3477,  -31.8698,   58.0286,  ...,  -34.4822,  -66.1560,
           36.3165],
        [  28.4368,   29.2973,   86.1600,  ..., -113.5144,  -91.9907,
           31.7262]], device='cuda:0', grad_fn=<AddBackward0>)

In [10]:
y_embedding

tensor([[  1.0275,   5.2332,   4.9076,  ...,  -0.1102,  -4.6409,   9.6365],
        [ -3.7336,   3.8080,  -3.3415,  ...,   6.5349,  -3.4549,  12.3156],
        [-11.5693,   6.7365,  -1.9683,  ...,  -3.5168,  -0.9344,   5.0738],
        ...,
        [  6.4145,   0.8702,   1.1931,  ...,   1.0504,  -7.6094,   6.8339],
        [ -5.8024,  -0.3618,  -0.1412,  ...,  -0.3977,  -3.0978,   5.2975],
        [ -3.4887,   0.1085,   1.1460,  ...,  -3.2997,  -4.6300,   3.2439]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [11]:
x_embedding

tensor([[ 3.2541,  0.9079,  2.0818,  ..., -0.4530, -0.6870, 10.7181],
        [-4.7226,  2.0234,  3.6004,  ..., -7.1243, -5.1279,  7.1974],
        [-0.2887, -1.2837, 11.8498,  ..., -2.4560,  0.2970,  4.6080],
        ...,
        [ 5.3946, -1.4264,  3.0134,  ...,  4.3871, -0.7321,  1.6377],
        [-5.1021,  2.7773,  3.5497,  ...,  1.3156, -6.4151,  7.3006],
        [-5.5785,  2.6395,  2.6669,  ...,  1.7412, -5.4692,  8.5676]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
x_delta_hat

tensor([[  51.2291,  -93.8777,   67.3637,  ...,  -48.3670,  -35.8315,
           35.6972],
        [  31.9506,  -89.4450,   59.7837,  ...,  -90.8423,  -70.3604,
           44.9571],
        [  38.3420,  -81.8791,   54.3903,  ...,  -79.0527,  -53.9882,
           40.6473],
        ...,
        [  67.1868, -110.0594,   83.8629,  ...,  -42.5928,  -26.6214,
           46.3152],
        [  24.4498,  -34.6471,   54.4789,  ...,  -35.7978,  -59.7409,
           29.0159],
        [  34.0153,   26.6578,   83.4931,  ..., -115.2556,  -86.5215,
           23.1586]], device='cuda:0', grad_fn=<AddmmBackward>)

# Run all model, get some predictions

In [13]:
for idx, i in enumerate(val_data_loader):
    if idx == 1:
        X = (MolGraph(i[0]))
        Y = (MolGraph(i[1]))
        break

In [14]:
x_embedding, x_delta_hat = molopt.forward(X)
results_logits = molopt_decoder.forward(x_embedding + x_delta_hat, X)
results_labels = molopt_decoder.discretize(*results_logits)

In [15]:
symbols_logits, charges_logits, bonds_logits = results_logits
symbols_labels, charges_labels, bonds_labels = results_labels

In [16]:
symbols_labels

tensor([2, 0, 0,  ..., 0, 0, 0], device='cuda:0', grad_fn=<NotImplemented>)

In [17]:
bonds_labels_arr = bonds_labels.cpu().numpy()

In [18]:
np.unique(bonds_labels_arr, return_counts=True)

(array([0, 1, 2, 3, 4]), array([  302,    90,     4,   373, 21884]))

In [19]:
target_symbols, target_bonds = encode_target(Y)
target_charges = target_symbols
target_symbols_labels, target_charges_labels, target_bonds_labels = molopt_decoder.discretize(target_symbols, target_charges, target_bonds)

In [20]:
print (np.unique(target_symbols_labels.cpu().numpy(), return_counts=True))
print (np.unique(symbols_labels.cpu().numpy(), return_counts=True))

(array([0, 1, 2, 3, 4, 7, 8]), array([792, 113,  92,   7,  22,   5,   6]))
(array([0, 1, 2, 3, 4, 7, 8]), array([805, 108,  91,   8,  16,   5,   4]))


In [21]:
print (np.unique(target_bonds_labels.cpu().numpy(), return_counts=True))
print (np.unique(bonds_labels.cpu().numpy(), return_counts=True))

(array([0, 1, 2, 3, 4]), array([ 1164,   106,     4,   960, 20419]))
(array([0, 1, 2, 3, 4]), array([  302,    90,     4,   373, 21884]))


In [22]:
symbols_logits

tensor([[ 1.8041e+01,  1.7211e+01,  2.5616e+01,  ..., -1.5842e+01,
          7.7784e+00,  9.1317e+00],
        [ 3.0312e+01,  1.6935e+01,  2.0793e+01,  ..., -1.1996e+01,
          1.0808e+01,  9.5764e+00],
        [ 3.2457e+01,  1.8741e+01,  1.5606e+01,  ...,  4.1714e+00,
          6.2777e+00,  1.7968e+01],
        ...,
        [ 2.7157e+01,  1.7626e+01,  1.0654e+01,  ..., -4.2492e+00,
          1.9643e+00,  4.4446e+00],
        [ 2.0630e+01,  1.4614e+01,  1.2508e+01,  ..., -5.7855e+00,
         -1.3627e-02,  6.4817e+00],
        [ 3.2913e+01,  1.8050e+01,  1.4593e+01,  ...,  7.8303e-01,
          4.1754e+00,  1.5034e+01]], device='cuda:0', grad_fn=<AddmmBackward>)

In [23]:
symbols_labels

tensor([2, 0, 0,  ..., 0, 0, 0], device='cuda:0', grad_fn=<NotImplemented>)

In [24]:
os.getcwd()

'/home/octav/gitrepos/tum-thesis'

# Visualize

In [25]:
from tensorboardX import SummaryWriter
tb_writer = SummaryWriter(logdir = "mol_opt/logs")

mol_drawer = MolDrawer(tb_writer, SYMBOLS, BOND_TYPES, FORMAL_CHARGES)

In [32]:
#preprocess
from rdkit import Chem
initial_smiles = [Chem.MolToSmiles(x) for x in X.rd_mols]
target_smiles = [Chem.MolToSmiles(y) for y in Y.rd_mols]

labels = (symbols_labels, charges_labels, bonds_labels)
# labels = (target_symbols_labels, target_charges_labels, target_bonds_labels)
prediction = (labels, None, Y.scope)

In [36]:
mol_drawer.visualize_batch(prediction, target_smiles, 1, initial_smiles)

In [31]:
import io

In [ ]:
bonds_labels